In [1]:
from importlib import reload
import json
import pretty_midi
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 422091078071127324
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13336856610618758248
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5588779008
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11742072890166847630
physical_device_desc: "device: 0, name: Tesla K20Xm, pci bus id: 0000:00:06.0, compute capability: 3.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 152799329061713978
physical_device_desc: "device: XLA_GPU device"
]


In [10]:
try:
    reload(midiMethods)
    from modules.midiMethods import *
except:
    print('reload failed')
    import modules.midiMethods as midiMethods
    from modules.midiMethods import *
try:
    reload(dataMethods)
    from modules.dataMethods import *
except:
    print('reload failed')
    import modules.dataMethods as dataMethods
    from modules.dataMethods import *
import modules.mlClasses as mlClasses 


In [27]:
import modules.models as models
reload(models)
import modules.mlClasses as mlClasses
reload(mlClasses)

<module 'modules.mlClasses' from '/home/ubuntu/music-ml/380-music-representation/modules/mlClasses.py'>

In [175]:
# test out my function for going to note_bin representation
pm = pretty_midi.PrettyMIDI('midi/basic_test.mid')
noteB2 = pm2note_bin(pm)
print(noteB2)

[[27, 0, 0, 3, 19, 27], [0, 0, 0, 3, 20, 28], [5, 3, 35, 3, 14, 25], [32, 0, 0, 3, 16, 26], [2, 3, 27, 0, 21, 17], [34, 0, 0, 1, 13, 19], [3, 0, 42, 0, 20, 11], [5, 0, 38, 0, 21, 11], [32, 0, 3, 1, 20, 13], [2, 0, 42, 0, 28, 9], [7, 0, 59, 2, 16, 8], [31, 0, 2, 2, 15, 5]]


In [75]:
# if needed, read in filenames...
maestro = pd.read_csv('training_data/maestro-v2.0.0withPeriod.csv', index_col=0)
filenames = list(maestro['midi_filename'])


In [ ]:
# or process data set
data_path = 'C:/Users\Andrew/Documents/mlprojects/Datasets/MaestroV2.00/maestro-v2.0.0/'
examples = files2note_bin_examples(data_path, skip=1)
print(len(examples))

In [322]:
# store examples in json format
with open('training_data/training_data_note_bin_complete.json', 'w') as f:
    json.dump(examples, f)

In [20]:
# or read them in
with open('training_data/training_data_note_bin_complete.json', 'r') as f:
    examples = json.load(f)

In [28]:
# build simple model
# excellent example of recurrent model here https://www.tensorflow.org/tutorials/text/text_generation
hidden_state = 256
model2 = models.create_model1(hidden_state_size=hidden_state)
training_generator = mlClasses.DataGenerator(examples)

Model: "1layerLSTM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 6)]     0                                            
__________________________________________________________________________________________________
lambda_18 (Lambda)              (None, 128, 88)      0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_19 (Lambda)              (None, 128, 10)      0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_20 (Lambda)              (None, 128, 60)      0           input_4[0][0]                    
_________________________________________________________________________________________

In [29]:
model2.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# checkpoint = tf.keras.callbacks.ModelCheckpoint("weights/model1/{epoch:02d}-{train_loss:.2f}.hdf5", monitor='train_loss', verbose=2, save_best_only=True, save_weights_only=True)
model2.fit_generator(training_generator, epochs=10)
model2.save_weights('models/model2_10epochs256state.h5')

Epoch 1/10


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: Can not squeeze dim[1], expected a dimension of 1, got 128
	 [[{{node metrics_4/dense_22_acc/Squeeze}}]]
	 [[loss_2/Identity_2/_1237]]
  (1) Invalid argument: Can not squeeze dim[1], expected a dimension of 1, got 128
	 [[{{node metrics_4/dense_22_acc/Squeeze}}]]
0 successful operations.
0 derived errors ignored.

In [310]:
[Y[:,:,i] for i in range(6)][4]

array([[28, 13, 13, ..., 13, 15, 14],
       [14, 15, 22, ..., 11,  4, 25],
       [17, 22, 18, ..., 18, 17, 10],
       ...,
       [11, 11, 29, ..., 13,  3,  2],
       [ 0, 29, 28, ..., 17, 11,  9],
       [ 9,  8,  3, ..., 29, 29, 29]])

In [465]:
# This works just like this:
try:
    reload(modules.models)
except:
    print('reload failed')
    import modules.models as models

In [22]:
# tf.train.latest_checkpoint('weights\first_attempt')

prediction_model = models.create_model1(batch_size=1, stateful=True, hidden_state_size=hidden_state)

Model: "1layerLSTM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(1, 128, 6)]        0                                            
__________________________________________________________________________________________________
lambda_6 (Lambda)               (1, 128, 88)         0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_7 (Lambda)               (1, 128, 10)         0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_8 (Lambda)               (1, 128, 60)         0           input_2[0][0]                    
_________________________________________________________________________________________

In [466]:
prediction_model.load_weights('weights/model1_2epochs.h5')
new_seq = models.generate_music(prediction_model)
ns = [[int(a) for a in note] for note in new_seq]
print(ns)
pm = note_bin2pm(ns)
pm.write('midi/first_Tnotes1_0.2b.mid')

tf.Tensor([[[34  0  3  3  3 16]]], shape=(1, 1, 6), dtype=int32)
[[87, 0, 1, 11, 27, 1], [68, 0, 42, 4, 20, 31], [27, 0, 55, 4, 7, 25], [71, 0, 1, 4, 7, 23], [39, 0, 9, 15, 3, 17], [39, 0, 43, 6, 19, 11], [30, 0, 46, 6, 19, 13], [64, 0, 14, 6, 3, 30], [0, 0, 11, 6, 1, 1], [42, 9, 2, 5, 25, 0], [54, 0, 41, 5, 22, 11], [42, 0, 6, 5, 19, 2], [56, 4, 5, 4, 18, 26], [61, 2, 23, 6, 14, 20], [54, 0, 56, 10, 2, 27], [18, 0, 50, 1, 27, 19], [31, 0, 5, 12, 14, 4], [7, 0, 47, 14, 17, 29], [51, 1, 18, 14, 9, 12], [85, 0, 0, 14, 2, 20], [48, 0, 32, 13, 16, 29], [52, 0, 27, 13, 27, 9], [55, 2, 36, 13, 1, 9], [76, 0, 41, 13, 11, 10], [13, 9, 24, 13, 2, 6], [73, 0, 51, 12, 21, 6], [5, 0, 39, 12, 9, 11], [76, 3, 10, 12, 3, 17], [85, 0, 18, 17, 17, 17], [43, 0, 54, 2, 14, 30], [16, 0, 18, 2, 22, 17], [33, 0, 42, 2, 13, 18], [65, 0, 46, 2, 10, 11], [77, 0, 19, 2, 25, 15], [83, 7, 31, 17, 17, 29], [3, 1, 5, 14, 21, 31], [64, 0, 57, 15, 2, 16], [73, 1, 20, 14, 29, 8], [49, 0, 16, 13, 3, 9], [81, 0, 21, 1, 

[[21, 0, 1, 3, 2, 17], [52, 0, 0, 3, 2, 16], [43, 0, 0, 3, 2, 15], [46, 0, 0, 3, 2, 15], [53, 0, 0, 3, 2, 16], [41, 0, 1, 3, 2, 16], [38, 0, 1, 3, 2, 15], [35, 0, 1, 3, 2, 16], [45, 0, 1, 3, 2, 16], [46, 0, 1, 3, 2, 16], [53, 0, 0, 3, 2, 17], [44, 0, 0, 3, 2, 17], [39, 0, 0, 3, 2, 18], [36, 0, 0, 3, 2, 18], [39, 0, 0, 3, 2, 17], [32, 0, 0, 3, 2, 15], [44, 0, 0, 3, 2, 16], [36, 0, 0, 3, 2, 17], [34, 0, 0, 0, 2, 17], [46, 0, 0, 0, 2, 18], [69, 0, 0, 0, 2, 16], [60, 0, 1, 0, 2, 16], [36, 0, 1, 0, 2, 14], [48, 0, 0, 0, 2, 14], [42, 0, 0, 0, 2, 15], [50, 0, 0, 0, 2, 16], [50, 0, 1, 0, 2, 17], [38, 0, 0, 0, 2, 17], [36, 0, 0, 0, 2, 16], [32, 0, 1, 0, 2, 17], [41, 0, 0, 0, 2, 16], [30, 0, 0, 0, 2, 16], [34, 0, 0, 0, 2, 16], [40, 0, 0, 0, 2, 17], [38, 0, 0, 0, 2, 17], [50, 0, 0, 0, 2, 16], [51, 0, 0, 0, 2, 17], [49, 0, 0, 0, 2, 17], [27, 0, 0, 0, 2, 16], [39, 0, 0, 0, 2, 17], [66, 0, 0, 0, 2, 18], [55, 0, 0, 0, 2, 17], [43, 0, 0, 0, 2, 16], [50, 0, 0, 0, 2, 16], [56, 0, 0, 0, 2, 15], [55, 0, 1